In [ ]:
import sys
import os
sys.path.append(os.path.join(sys.path[0], '../..'))

from pylamarck.spaces.permutations.test_functions_tsp import RandomCities,\
    visualize, tsp_swap_neighbourhood
from pylamarck.logging import EvaluationLogger, logme, goal_val_plot
from pylamarck.termination import TimeLimit, MaxSteps
from pylamarck.algorithms.random_sampling import RandomSampling
from pylamarck.algorithms.random_walk import RandomWalk
from pylamarck.algorithms.hill_climbing import HillClimbing
from pylamarck.spaces.permutations.production import PermutationSingleSwap
from pylamarck.production import ConstantSearch
from pylamarck.spaces.permutations.production import RandomPermutation
from pylamarck.spaces.bits.production import RandomBitSequence, BitFlips
from pylamarck.algorithms.simulated_annealing import SimulatedAnnealing,\
    ExponentialSchedule, LogarithmicSchedule, \
    PolynomialSchedule, temperature_schedule_plot
from pylamarck.spaces.bits.test_functions_bits import MaxSAT,\
    sat_neighbourhood
from pylamarck.algorithms.tabu_search import TabuSearch,\
    SimpleTabuSearchNeighbourhood


In [ ]:
def test1():
    city_number = 10

    el_rs = EvaluationLogger("Random sampling")
    el_hc = EvaluationLogger("Hill climbing")
    el_rw = EvaluationLogger("Random walk")

    cityfun = RandomCities(city_number)

    rp = RandomPermutation(city_number)
    search = RandomSampling(rp,
                            TimeLimit(1) | MaxSteps(5000))
    xrs = search.solve(logme(el_rs)(cityfun))
    print("Random sampling: ", xrs)

    pss = PermutationSingleSwap(city_number)
    search = HillClimbing(ConstantSearch([i for i in range(city_number)]),
                          pss,
                          TimeLimit(1) | MaxSteps(5000))
    xhc = search.solve(logme(el_hc)(cityfun))
    print("Hill climbing: ", xhc)

    search = RandomWalk(ConstantSearch([i for i in range(city_number)]),
                        pss,
                        TimeLimit(1) | MaxSteps(5000))
    xrw = search.solve(logme(el_rw)(cityfun))
    print("Random walk: ", xrw)

    goal_val_plot([el_rs, el_hc, el_rw], mintoi=True, xtimestep='time', fmin=0)
    visualize(cityfun, [{"label": "Random sampling", "perm": xrs.g},
                        {"label": "Hill climbing", "perm": xhc.g},
                        {"label": "Random walk", "perm": xrw.g}])


test1()


In [ ]:

def test2():
    schedules = [{"label": "exponential", "schedule": ExponentialSchedule(1.0, 0.05)},
                 {"label": "logarithmic", "schedule": LogarithmicSchedule(1.0)},
                 {"label": "polynomial", "schedule": PolynomialSchedule(1.0, 2, 500)}]

    city_number = 50
    max_time = 1 # in seconds
    max_steps = 10000

    el_rs = EvaluationLogger("Random sampling")
    el_hc = EvaluationLogger("Hill climbing")
    el_rw = EvaluationLogger("Random walk")
    el_sa_list = []

    cityfun = RandomCities(city_number)

    rp = RandomPermutation(city_number)
    search = RandomSampling(rp,
                            TimeLimit(max_time) | MaxSteps(max_steps))
    xrs = search.solve(logme(el_rs)(cityfun))
    print("Random sampling: ", xrs)

    pss = PermutationSingleSwap(city_number)
    nso = ConstantSearch([i for i in range(city_number)])
    search = HillClimbing(nso,
                          pss,
                          TimeLimit(max_time) | MaxSteps(max_steps))
    xhc = search.solve(logme(el_hc)(cityfun))
    print("Hill climbing: ", xhc)

    search = RandomWalk(nso,
                        pss,
                        TimeLimit(max_time) | MaxSteps(max_steps))
    xrw = search.solve(logme(el_rw)(cityfun))
    print("Random walk: ", xrw)

    to_plot = [{"label": "Random sampling", "perm": xrs.g},
               {"label": "Hill climbing", "perm": xhc.g},
               {"label": "Random walk", "perm": xrw.g}]

    # to_plot.clear()
    for s in schedules:
        search = SimulatedAnnealing(nso, pss, TimeLimit(max_time) | MaxSteps(max_steps), s["schedule"])
        el = EvaluationLogger("Simulated annealing " + s["label"])
        el_sa_list.append(el)
        xsa = search.solve(logme(el)(cityfun))
        print(s["label"] + ": " + str(xsa))
        to_plot.append({"label": "Simulated annealing " + s["label"], "perm": xsa.g})

    goal_val_plot([el_rs, el_hc, el_rw] + el_sa_list, mintoi=True, xtimestep='time', fmin=0)
    visualize(cityfun, [min(to_plot, key=lambda x: cityfun(x["perm"]))])

    temperature_schedule_plot(schedules, 1000)


test2()


In [ ]:

def test3():
    # Tabu search
    # MAX-SAT problem
    n_vars = 50
    n_conj = 30
    n_disj = 5
    max_time = 5
    max_steps = 50000

    nso_rand = RandomBitSequence(n_vars)
    nso_const = ConstantSearch([True for _ in range(n_vars)])
    el_tabu = EvaluationLogger("tabu")

    msat = MaxSAT(n_vars, n_conj, n_disj)
    tabu_max_len = 30  # the result is very sensitive to this parameter
    neighbourhood_search = SimpleTabuSearchNeighbourhood(sat_neighbourhood)
    search = TabuSearch(nso_rand,
                        neighbourhood_search,
                        TimeLimit(max_time) | MaxSteps(max_steps),
                        tabu_max_len)
    xts = search.solve(logme(el_tabu)(msat))
    print(xts)

    el_sa = EvaluationLogger("simulated annealing")
    schedule = PolynomialSchedule(1.0, 2, 500)
    uso = BitFlips(2)
    search = SimulatedAnnealing(nso_rand,
                                uso,
                                TimeLimit(max_time) | MaxSteps(max_steps),
                                schedule)
    xsa = search.solve(logme(el_sa)(msat))
    print(xsa)

    goal_val_plot([el_tabu, el_sa], mintoi=True, xtimestep='time', fmin=0, plot_logarithm=False)
    
    
test3()


In [ ]:

def test4():
    # Travelling Salesman Problem
    city_number = 30
    max_time = 20  # in seconds
    max_steps = 1000000

    el_hc = EvaluationLogger("Hill climbing")
    el_tabu_swap = EvaluationLogger("tabu swap")
    el_tabu_reverse = EvaluationLogger("tabu reverse")

    cityfun = RandomCities(city_number)

    pss = PermutationSingleSwap(city_number)
    # nso = ConstantSearch([i for i in range(city_number)])
    nso = RandomPermutation(city_number)
    search = HillClimbing(nso,
                          pss,
                          TimeLimit(max_time) | MaxSteps(max_steps))
    xhc = search.solve(logme(el_hc)(cityfun))
    print("Hill climbing: ", xhc)

    tabu_max_len = 500  # the result is very sensitive to this parameter

    neighbourhood_search = SimpleTabuSearchNeighbourhood(tsp_swap_neighbourhood)

    search = TabuSearch(nso,
                        neighbourhood_search,
                        TimeLimit(max_time) | MaxSteps(max_steps),
                        tabu_max_len)
    xts = search.solve(logme(el_tabu_swap)(cityfun))
    print("Tabu search swap: ", xts)

    search = TabuSearch(nso,
                        neighbourhood_search,
                        TimeLimit(max_time) | MaxSteps(max_steps),
                        tabu_max_len)
    xtr = search.solve(logme(el_tabu_reverse)(cityfun))
    print("Tabu search reverse: ", xtr)

    el_sa = EvaluationLogger("simulated annealing")
    schedule = PolynomialSchedule(1.0, 2, 500)
    pss = PermutationSingleSwap(city_number)
    search = SimulatedAnnealing(nso,
                                pss,
                                TimeLimit(max_time) | MaxSteps(max_steps),
                                schedule)
    xsa = search.solve(logme(el_sa)(cityfun))
    print("Simulated Annealing: ", xsa)

    to_plot = [{"label": "tabu swap", "perm": xts.g},
               {"label": "tabu reverse", "perm": xtr.g},
               {"label": "simulated annealing", "perm": xsa.g},
               {"label": "hill climbing", "perm": xhc.g}]
    goal_val_plot([el_tabu_swap, el_tabu_reverse, el_sa, el_hc],
                  mintoi=True,
                  xtimestep='time',
                  fmin=0,
                  plot_logarithm=False)
    visualize(cityfun, [min(to_plot, key=lambda x: cityfun(x["perm"]))])


test4()